In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np


/home/student/my_env/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda:0"
                      if torch.cuda.is_available()
                      else "cpu")

In [3]:
# Data setting

from sklearn import datasets
from sklearn.model_selection import train_test_split

iris = datasets.load_iris()
xtrain, xtest, ytrain, ytest = train_test_split(iris.data, iris.target, test_size=0.5)

In [5]:
xtrain = torch.from_numpy(xtrain).type("torch.FloatTensor")
ytrain = torch.from_numpy(ytrain).type('torch.LongTensor')
xtest = torch.from_numpy(xtest).type('torch.FloatTensor')
ytest = torch.from_numpy(ytest).type('torch.LongTensor')

In [6]:
xtrain = xtrain.to(device)
ytrain = ytrain.to(device)
xtest = xtest.to(device)
ytest = ytest.to(device)

In [9]:
# Define model

class MyIris(nn.Module):
    def __init__(self):
        super(MyIris,self).__init__()
        self.l1=nn.Linear(4,6)
        self.l2=nn.Linear(6,3)
    def forward(self,x):
        h1=torch.sigmoid(self.l1(x))
        h2=self.l2(h1)
        return h2

In [10]:
# model generate, optimizer and criterion setting

model= MyIris().to(device)
optimizer = optim.SGD(model.parameters(),lr=0.1)
criterion = nn.CrossEntropyLoss()

In [11]:
#Learn

n = 75    ##  データのサイズ
bs = 25   ##  バッチのサイズ

model.train()
for i in range(1000):
    idx = np.random.permutation(n)
    for j in range(0,n,bs):
        xtm = xtrain[idx[j:(j+bs) if (j+bs)<n else n]]
        ytm = ytrain[idx[j:(j+bs) if (j+bs) < n else n]]
        output = model(xtm)
        loss = criterion(output,ytm)
        print(i,j,loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

0 0 0.9815385937690735
0 25 1.1578748226165771
0 50 1.216102957725525
1 0 1.057498574256897
1 25 1.1156697273254395
1 50 1.0772608518600464
2 0 1.107243299484253
2 25 1.0283288955688477
2 50 1.0799665451049805
3 0 1.049027442932129
3 25 1.0889016389846802
3 50 1.0399799346923828
4 0 1.0533901453018188
4 25 1.0373131036758423
4 50 1.0614650249481201
5 0 1.0535835027694702
5 25 1.030823826789856
5 50 1.0378848314285278
6 0 1.0309261083602905
6 25 1.0485501289367676
6 50 1.026870608329773
7 0 1.020190954208374
7 25 1.0290545225143433
7 50 1.024576187133789
8 0 1.0216212272644043
8 25 1.038154125213623
8 50 1.0002578496932983
9 0 1.0092995166778564
9 25 0.9910755753517151
9 50 1.0260777473449707
10 0 1.0110516548156738
10 25 0.9823975563049316
10 50 0.9953340888023376
11 0 0.9757039546966553
11 25 0.994960606098175
11 50 0.9855908155441284
12 0 0.957103431224823
12 25 0.975391685962677
12 50 0.9894105792045593
13 0 0.9956183433532715
13 25 0.9428578019142151
13 50 0.951578676700592
14 0 0.

In [12]:
# torch.save(model.state_dict(),'myiris.model')     ## モデルの保存
# model.load_state_dict(torch.load('myiris.model')) ## モデルの呼び出し

In [14]:
#Test

model.eval()
with torch.no_grad():
    output1= model(xtest)
    ans = torch.argmax(output1,1)
    print(((ytest == ans).sum().float() / len(ans) ).item() )

0.9733333587646484
